# Generate input for negative class

In [1]:
import os
from tqdm import tqdm
from lxml import etree
import json
import random 
import string
from perdido.geoparser import Geoparser
from utils_functions import load_lexicon, load_edda_dataframe, run_perdido, get_term_occurrences_from_ene, segment_sentences, get_lexicon_occurrences, get_ngrams_wt_term_outside_ene

[nltk_data] Downloading package punkt to /Users/lmoncla/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 1. Load lexicon

The lexicon is created with the notebook `generate_input_positive_class.ipynb`. (TODO: make an independant script)


In [2]:
lexicon_filename =  os.path.join('output', 'Traitement_Texte_pivot_lexicon.json')

frequency_dict_geo_tt = load_lexicon(lexicon_filename)

In [3]:
len(frequency_dict_geo_tt)

190

# 2. Generate negative ngrams

## 2.1 Load EDdA dataset

Load the csv of EDdA dataset as a dataframe. Article from the Encyclopédie will be used to generate negative ngrams with words from the lexicon as pivot.

In [ ]:
edda_dataset_path = '/Users/lmoncla/Nextcloud-LIRIS/GEODE/GEODE - Partage consortium/Corpus/EDdA/EDdA_dataset_articles_superdomainBERT_230327.tsv'
data = load_edda_dataframe(edda_dataset_path, 'Philosophie')
data.head()

In [ ]:
data.shape

## 2.1 Build batches of sentences

In [ ]:
#sentences_per_article = data['content'].apply(segment_sentences)
df = data.head()
sentences_per_article = df['content'].apply(segment_sentences)

all_sentences = [sentence for sentences in sentences_per_article for sentence in sentences]

# Print the list of sentences
for sentence in all_sentences:
    print(sentence)

# build batches of sentences
batch_size = 50
batches = [all_sentences[i:i + batch_size] for i in range(0, len(all_sentences), batch_size)]

# turn each batch into a concatenate string
batch_strings = [' '.join(batch) for batch in batches]

In [ ]:
batch_strings

## 2.2 Run Perdido on the batches of sentences

In [4]:
perdido_dir = './output/Philosophie'

In [ ]:
geoparser = Geoparser(version='Standard', sources=['wiki_gaz'])

for batch in tqdm(batch_strings):
    # generate a random string of 10 characters
    filename = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))
    doc = run_perdido(batch, geoparser)
    try:
        doc.to_xml(os.path.join(perdido_dir, filename + '.xml'))
    except:
        print('Error', filename)


## 2.3 Get frequency of terms from the lexicon in the ENE 


* Regarde pour chaque terme du lexique (TT) sa fréquence dans les ENE des articles de Philosophie. 

In [5]:
terms = []
for doc in tqdm(sorted(os.listdir(perdido_dir))):
    terms.extend(get_term_occurrences_from_ene(os.path.join(perdido_dir,doc)))

get_lexicon_occurrences(terms, frequency_dict_geo_tt.keys())

100%|██████████| 6/6 [00:00<00:00, 585.52it/s]


{'chemin': 0,
 'lieu-dit': 0,
 'chemins': 0,
 'plaine': 0,
 'sommet de la ligne': 0,
 'sommet': 0,
 'sentier': 0,
 'champ': 0,
 'col': 0,
 'rocher': 0,
 'sentiers': 0,
 'route': 0,
 'piste': 0,
 'lac': 0,
 'parking': 0,
 'intersection': 0,
 'auberge': 0,
 'port de chaussures': 0,
 'port': 0,
 'massif': 0,
 'vallée': 0,
 'crête': 0,
 'sommets': 0,
 'grotte': 0,
 'gare': 0,
 'place': 0,
 'ruisseau': 0,
 'parking du lac': 0,
 'rochers': 0,
 'source du lac': 0,
 'source': 0,
 'parking de la route': 0,
 'ferme': 0,
 'refuge': 0,
 'chalet': 0,
 'auberge du lac': 0,
 'ruines': 0,
 'parking du château': 0,
 'château': 0,
 'barrage': 0,
 'villages': 0,
 'cascades': 0,
 'châteaux': 0,
 'abbaye': 0,
 'rue': 4,
 'rivière': 0,
 'carrefour': 0,
 'cour': 0,
 'route des châteaux': 0,
 'village': 0,
 'parking au pied': 0,
 'galeries': 0,
 "parking de l' auberge": 0,
 'pistes': 0,
 'versant': 0,
 'passages': 0,
 'sources': 0,
 'moulin': 0,
 '': 0,
 'étang': 0,
 "canal d' amenée": 0,
 'canal': 0,
 'mur':

## 2.4 Build ngrams


* Regarde pour chaque terme du lexique (TT) sa fréquence en dehors des ENE des articles de Philosophie. 

* construire les grams, fonction : get_ngrams_wt_term_outside_ene()

In [6]:
ngrams = []
for doc in tqdm(sorted(os.listdir(perdido_dir))):
    ngrams.extend(get_ngrams_wt_term_outside_ene(os.path.join(perdido_dir,doc), frequency_dict_geo_tt, 1))

100%|██████████| 6/6 [00:00<00:00, 609.49it/s]

Chez BRIASSON , [ rue ] Saint Jacques , 
l' aîné , [ rue ] Saint Jacques , 
du Roy , [ rue ] de la Harpe 
. DURAND , [ rue ] Saint Jacques , 
surtout par le [ passage ] d' Eusebe , 
& bref dans [ place ] . Il est 
long , au [ lieu ] de l' s 
: ainsi au [ lieu ] d' écrire mastin 
Suivre à la [ piste ] . Faire le 
I. Dans son [ état ] simple : 1° 
sorte qu' au [ lieu ] de dire à 
au pluriel au [ lieu ] de dire à 
Puiser à la [ source ] . Prendre garde 
Adverbe ) au [ lieu ] que la préposition 
Grec ont donné [ lieu ] à ce vers 


In [9]:
pivots = [ngram['pivot'] for ngram in ngrams]
pivots

['rue',
 'rue',
 'rue',
 'rue',
 'passage',
 'place',
 'lieu',
 'lieu',
 'piste',
 'état',
 'lieu',
 'lieu',
 'source',
 'lieu',
 'lieu']

In [10]:
get_lexicon_occurrences(pivots, frequency_dict_geo_tt.keys())

{'chemin': 0,
 'lieu-dit': 0,
 'chemins': 0,
 'plaine': 0,
 'sommet de la ligne': 0,
 'sommet': 0,
 'sentier': 0,
 'champ': 0,
 'col': 0,
 'rocher': 0,
 'sentiers': 0,
 'route': 0,
 'piste': 1,
 'lac': 0,
 'parking': 0,
 'intersection': 0,
 'auberge': 0,
 'port de chaussures': 0,
 'port': 0,
 'massif': 0,
 'vallée': 0,
 'crête': 0,
 'sommets': 0,
 'grotte': 0,
 'gare': 0,
 'place': 1,
 'ruisseau': 0,
 'parking du lac': 0,
 'rochers': 0,
 'source du lac': 0,
 'source': 1,
 'parking de la route': 0,
 'ferme': 0,
 'refuge': 0,
 'chalet': 0,
 'auberge du lac': 0,
 'ruines': 0,
 'parking du château': 0,
 'château': 0,
 'barrage': 0,
 'villages': 0,
 'cascades': 0,
 'châteaux': 0,
 'abbaye': 0,
 'rue': 4,
 'rivière': 0,
 'carrefour': 0,
 'cour': 0,
 'route des châteaux': 0,
 'village': 0,
 'parking au pied': 0,
 'galeries': 0,
 "parking de l' auberge": 0,
 'pistes': 0,
 'versant': 0,
 'passages': 0,
 'sources': 0,
 'moulin': 0,
 '': 0,
 'étang': 0,
 "canal d' amenée": 0,
 'canal': 0,
 'mur':